# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [1]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64, 'wind_speed': pl.Float64})

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-349231014.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [2]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)


shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

> Add blockquote



In [3]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

1.   List item
2.   List item



In [4]:
# Write your SQL query here
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [5]:
# Write your SQL query here
result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin
""")

print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
│ EWR    ┆ 15.107954     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [6]:
# First, let's check what columns are available
result = ctx.execute("""
SELECT month, COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [7]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    carrier,
    AVG(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS on_time_rate
FROM flights
GROUP BY carrier
""")

print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ AA      ┆ 0.824376     │
│ AS      ┆ 0.865546     │
│ OO      ┆ 0.71875      │
│ HA      ┆ 0.929825     │
│ VX      ┆ 0.821        │
│ …       ┆ …            │
│ EV      ┆ 0.659221     │
│ WN      ┆ 0.719593     │
│ FL      ┆ 0.716871     │
│ US      ┆ 0.849873     │
│ DL      ┆ 0.830742     │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [8]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight,
    f.origin,
    f.dest
FROM flights AS f  -- Use 'flights' table for flight details
JOIN airlines AS a  -- Join with 'airlines' table for airline names
ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [9]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    AVG(2013 - p.year) AS avg_plane_age
FROM flights AS f
JOIN planes AS p ON f.tailnum = p.tailnum
GROUP BY f.carrier
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ YV      ┆ 9.313758      │
│ DL      ┆ 16.372169     │
│ F9      ┆ 4.87874       │
│ EV      ┆ 11.308998     │
│ MQ      ┆ 35.319        │
│ …       ┆ …             │
│ B6      ┆ 6.686702      │
│ US      ┆ 9.103663      │
│ FL      ┆ 11.385829     │
│ HA      ┆ 1.548387      │
│ OO      ┆ 6.84375       │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [10]:
# Write your SQL query here
result = ctx.execute("""
SELECT f.*
FROM flights AS f
JOIN weather AS w
ON f.origin = w.origin
AND f.year = w.year
AND f.month = w.month
AND f.day = w.day
AND f.hour = w.hour -- Also join on hour for more precise weather conditions
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)

shape: (4_938, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 2217     ┆ … ┆ 1005     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 2326     ┆ … ┆ 2248     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1238     ┆ … ┆ 820      ┆ 11   ┆ 59     ┆ 2013-01-04 16:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1400     ┆ … ┆ 212      ┆ 13   ┆ 20     ┆ 2013-01-04 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1455     ┆ … ┆ 187      ┆ 14   ┆ 22     ┆ 2013-01-04 19:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [11]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS num_flights
FROM flights AS f  -- Use 'flights' table
JOIN planes AS p   -- Join with 'planes' table
ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────────┐
│ manufacturer                  ┆ model           ┆ num_flights │
│ ---                           ┆ ---             ┆ ---         │
│ str                           ┆ str             ┆ u32         │
╞═══════════════════════════════╪═════════════════╪═════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278       │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553       │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051       │
│ BOEING                        ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807       │
│ BOEING                        ┆ 737-7H4         ┆ 10389       │
│ BOEING                        ┆ 757-222         ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932     

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [12]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    origin,
    dest,
    COUNT(*) AS total_flights
FROM flights
GROUP BY origin, dest
ORDER BY total_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌────────┬──────┬───────────────┐
│ origin ┆ dest ┆ total_flights │
│ ---    ┆ ---  ┆ ---           │
│ str    ┆ str  ┆ u32           │
╞════════╪══════╪═══════════════╡
│ JFK    ┆ LAX  ┆ 11262         │
│ LGA    ┆ ATL  ┆ 10263         │
│ LGA    ┆ ORD  ┆ 8857          │
│ JFK    ┆ SFO  ┆ 8204          │
│ LGA    ┆ CLT  ┆ 6168          │
│ EWR    ┆ ORD  ┆ 6100          │
│ JFK    ┆ BOS  ┆ 5898          │
│ LGA    ┆ MIA  ┆ 5781          │
│ JFK    ┆ MCO  ┆ 5464          │
│ EWR    ┆ BOS  ┆ 5327          │
└────────┴──────┴───────────────┘


In [13]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    origin,
    dest,
    AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin, dest
ORDER BY avg_dep_delay DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌────────┬──────┬───────────────┐
│ origin ┆ dest ┆ avg_dep_delay │
│ ---    ┆ ---  ┆ ---           │
│ str    ┆ str  ┆ f64           │
╞════════╪══════╪═══════════════╡
│ EWR    ┆ LGA  ┆ null          │
│ EWR    ┆ TYS  ┆ 41.818471     │
│ EWR    ┆ CAE  ┆ 36.336842     │
│ EWR    ┆ TUL  ┆ 34.906355     │
│ LGA    ┆ SBN  ┆ 31.333333     │
│ EWR    ┆ OKC  ┆ 30.568807     │
│ LGA    ┆ BHM  ┆ 29.778598     │
│ LGA    ┆ CAE  ┆ 29.5          │
│ EWR    ┆ DSM  ┆ 29.328244     │
│ EWR    ┆ JAC  ┆ 28.7          │
└────────┴──────┴───────────────┘


## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [14]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)



SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
